# All Imports

In [ ]:
! python --version

In [ ]:
pip install -qqq lightning wandb

In [ ]:
 ! pip install -qqq ripser kmapper

In [ ]:
! pip install -qqq umap-learn

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import os
import glob
import time
import random
import numpy as np
import scipy as sp
import pandas as pd
print ("Numpy:",np.__version__ , "Scipy:",sp.__version__, "Pandas",pd.__version__)

# import tensorflow as tf
# from tensorflow.keras import datasets, layers, Sequential, Model, activations, initializers, regularizers, constraints
# from tensorflow.keras.layers import InputSpec
# import tensorflow.keras.backend as K
# from tensorflow.keras.utils import to_categorical
# from tensorflow.python.compiler.tensorrt import trt_convert as trt
# print ("TensorFlow:",tf.__version__)
# # import horovod.tensorflow as hvd
# print ("GPU List:", tf.config.list_physical_devices('GPU'))

import torch
# import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

# from pytorch_lightning import loggers as pl_loggers
# from pytorch_lightning.loggers import MLFlowLogger
# from pytorch_lightning.loggers import NeptuneLogger
import wandb
from pytorch_lightning.loggers import WandbLogger

# from pytorch_lightning.plugins import CheckpointIO
# from pytorch_lightning.strategies import SingleDeviceStrategy
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.cuda as cuda
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from datetime import datetime
from sklearn.model_selection import cross_val_score
print ("Pytorch Normal:", torch.__version__)
print ("Pytorch Lightning:", pl.__version__)

# import pyspark as ps
# from pyspark.mllib.stat import Statistics
# from pyspark.sql import SparkSession

# from mlflow import log_metric, log_param, log_artifacts

from IPython import display
from matplotlib import pyplot as plt
# import matplotlib.cm as cm
# from matplotlib.colors import ListedColormap
# import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
# from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_friedman1
# from sklearn.datasets import make_friedman2
# from sklearn.datasets import make_friedman3
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.cluster import AgglomerativeClustering
# from sklearn.preprocessing import StandardScaler
# from sklearn.cluster import KMeans
# from sklearn.utils import resample
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import normalize
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.decomposition import PCA

from math import *

import umap

from ripser import ripser
from ripser import Rips
from persim import plot_diagrams
# import ripserplusplus as rpp
import kmapper as km

# import cv2
# import PIL
# import imageio
# import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Useful Definitions

In [ ]:
def havrda_charvat_entropy(predictions, labels, parameter_a):
    """
    Calculate the Havrda-Charvat entropy between predicted values and labels.

    Args:
        predictions (np.ndarray or torch.Tensor): Predicted values (shape: [batch_size, num_classes]).
        labels (np.ndarray or torch.Tensor): Ground truth labels (shape: [batch_size, num_classes]).
        parameter_a (float, optional): Parameter for the Havrda-Charvat entropy. Default is 2.0.

    Returns:
        float: Average Havrda-Charvat entropy across the batch.
    """
    assert predictions.shape == labels.shape, "Predictions and labels must have the same shape."

    # Normalize predictions and labels to probabilities
    predictions_prob = np.clip(predictions.detach().cpu().numpy(), 1e-10, 1.0 - 1e-10)
    labels_prob = np.clip(labels.detach().cpu().numpy(), 1e-10, 1.0 - 1e-10)

    # Calculate entropy term for each class
    entropy_term = -np.sum(predictions_prob * np.log(labels_prob), axis=1)

    # Compute the overall Havrda-Charvat entropy
    hc_entropy = np.mean((1.0 - np.exp(-parameter_a * entropy_term)) / parameter_a)

    return hc_entropy

In [ ]:
def calc_dist(arr_a,arr_b, method:str):

    if method == "L1":
        distance = np.sum(np.absolute(arr_a - arr_b))
    if method == "L2":
        distance = (np.sum(np.square(arr_a-arr_b)))**0.5
    return distance

def k_twin(matrix_a:np.array,matrix_b:np.array, k:int, method:str):


    n_a = matrix_a.shape[0]
    n_b = matrix_b.shape[0]

    if n_a != n_b:
        print ("matrix dimention doesn't match")
        return

    dist_matrix_a = np.zeros((n_a, n_a))
    dist_matrix_b = np.zeros((n_b, n_b))

    for i in range (0,n_a):
        for j in range (0,n_a):
            dist_matrix_a[i,j] = calc_dist(matrix_a[i],matrix_a[j], method)
            np.fill_diagonal(dist_matrix_a, np.inf)
    # print (dist_matrix_a)

    for i in range (0,n_a):
        for j in range (0,n_a):
            dist_matrix_b[i,j] = calc_dist(matrix_b[i],matrix_b[j], method)
            np.fill_diagonal(dist_matrix_b, np.inf)
    # print (dist_matrix_b)

    k_closest_a = np.zeros((n_a,k))
    for i in range (0,n_a):
        k_closest_a[i] = np.argsort(dist_matrix_a[i])[:k]
    # print(k_closest_a)

    k_closest_b = np.zeros((n_b,k))
    for i in range (0,n_b):
        k_closest_b[i] = np.argsort(dist_matrix_b[i])[:k]
    # print(k_closest_b)

    total_count = 0
    for i in range (0, n_a ):
        # print(set(k_closest_a[i]))
        count = len(set(k_closest_a[i]).intersection(set(k_closest_b[i])))
        total_count += count
        # print(total_count)

    JI = (total_count/(n_a*k))

    return JI

In [ ]:
def Category_blind_MH(matrix, boundaries, fold:int):
    # Split the matrix column-wise at the given boundaries
    linkage_blocks = np.split(matrix, boundaries, axis=1)

    # Initialize a list to store the new splits
    whole_matrix = []

    for block in linkage_blocks:
        long_block = []
        for i in range (0,fold-1):
          block_new = block.copy()
          np.random.shuffle(block_new)
          long_block.append(block_new)

        long_block_final = np.vstack(long_block)
        whole_matrix.append(long_block_final)

    # Join the splits back into one matrix
    final_matrix = np.hstack(whole_matrix)
    final_matrix = np.vstack([matrix, final_matrix])

    return final_matrix

# Data and Hyperparameters

In [ ]:
# data_path =  /content/drive/MyDrive/KT/Data
x_train = np.load("/content/drive/MyDrive/KT/Data/KTR_2_2/DEAS/D_train_2_2.npy")
x_val = np.load("/content/drive/MyDrive/KT/Data/KTR_2_2/DEAS/D_val_2_2.npy")
x_test = np.load("/content/drive/MyDrive/KT/Data/KTR_2_2/DEAS/D_test_2_2.npy")

y_train= np.load("/content/drive/MyDrive/KT/Data/KTR_2_2/PSEUDO_SHARE/S_train_2_2.npy")
y_val = np.load("/content/drive/MyDrive/KT/Data/KTR_2_2/PSEUDO_SHARE/S_val_2_2.npy")
y_test = np.load("/content/drive/MyDrive/KT/Data/KTR_2_2/PSEUDO_SHARE/S_test_2_2.npy")


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda.is_available(),cuda.device_count()

(True, 1)

In [ ]:
# m0t0 is general without data augmentation
grad_clip = None
drop_out_rate = 0.0
num_epochs = 5000
batch_size = 256
l1_lamb = 0
l2_lamb = 1
pt_lamb = 0
hce_lamb = 0
tji_lamb = 0.1


In [ ]:
lr_m = 0.0002
b1_m = 0.5
b2_m = 0.999
lr_i = 0.0002
b1_i = 0.5
b2_i = 0.999

In [ ]:
Mapper_input_size,Mapper_output_size,Mapper_learning_rate = 56,31,1e-3
Inverter_input_size,Inverter_output_size,Inverter_learning_rate = 31,56,1e-3

In [ ]:
x_train_tt = torch.tensor(x_train,dtype=torch.float)#.unsqueeze(1)
x_val_tt = torch.tensor(x_val, dtype=torch.float)#.unsqueeze(1)
x_test_tt = torch.tensor(x_test, dtype=torch.float)#.unsqueeze(1)
y_train_tt = torch.tensor(np.array(y_train),dtype=torch.float)#.unsqueeze(1)
y_val_tt = torch.tensor(np.array(y_val), dtype=torch.float)#.unsqueeze(1)
y_test_tt = torch.tensor(np.array(y_test), dtype=torch.float)#.unsqueeze(1)

In [ ]:
train_dataset = TensorDataset(x_train_tt, y_train_tt)
val_dataset = TensorDataset(x_val_tt, y_val_tt)
test_dataset = TensorDataset(x_test_tt,y_test_tt)
training_loader = DataLoader(dataset= train_dataset, batch_size=batch_size, shuffle= True)
validation_loader = DataLoader(dataset= val_dataset, batch_size=batch_size, shuffle= False)
test_loader = DataLoader(dataset= test_dataset, batch_size=batch_size, shuffle= False)

# Architecture


In [ ]:
class Mapper (pl.LightningModule):
    def __init__(self, Mapper_input_size,Mapper_output_size,Mapper_learning_rate ):
        super(Mapper, self).__init__()
        self.Mapper_input_size = Mapper_input_size
        self.Mapper_output_size = Mapper_output_size

        # self.latent_size = latent_size
        self.Mapper_learning_rate = Mapper_learning_rate
        self.fc1 = nn.Linear(56, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6= nn.Linear(128,31)
        # self.fc4= nn.Linear(64,9)
        # self.fc3 = nn.Linear(128, 32)
        # self.fc4 = nn.Linear(32, 4)
        self.drop = nn.Dropout(p=0.3)

        self.bn1 = nn.BatchNorm1d(num_features=128, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn2 = nn.BatchNorm1d(num_features=256, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn3 = nn.BatchNorm1d(num_features=512, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn4 = nn.BatchNorm1d(num_features=256, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn5 = nn.BatchNorm1d(num_features=128, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn6 = nn.BatchNorm1d(num_features=31, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )

        self.prelu1 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu2 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu3 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu4 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu5 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu6 = nn.PReLU(num_parameters=1, init=0.25)


        # self.softmax = nn.Softmax(dim=-1)
        # self.tanh = nn.Tanh()
        # self.softmax = nn.Softmax(dim=1)


    def forward(self,x):
        x= torch.flatten(x, start_dim=1)
        # print (x.shape)
        x = self.fc1(x)
        x = self.prelu1(x)
        # x= self.drop(x)
        x= self.bn1(x)

        x = self.fc2(x)
        x = self.prelu2(x)
        # x= self.drop(x)
        x= self.bn2(x)

        x = self.fc3(x)
        x = self.prelu3(x)
        # x= self.drop(x)
        x= self.bn3(x)

        x = self.fc4(x)
        x = self.prelu4(x)
        # x= self.drop(x)
        x= self.bn4(x)

        x = self.fc5(x)
        x = self.prelu5(x)
        # x= self.drop(x)
        x= self.bn5(x)

        x = self.fc6(x)
        x = self.prelu6(x)
        # x= self.drop(x)
        # x= self.bn6(x)


        # x = self.fc4(x)
        # x = self.softmax(x)
        # print(x.shape)

        # x = self.tanh(x)

        return (x)

class Inverter (pl.LightningModule):
    def __init__(self, Inverter_input_size,Inverter_output_size,Inverter_learning_rate ):
        super(Inverter, self).__init__()
        self.Inverter_input_size = Inverter_input_size
        self.Inverter_output_size = Inverter_output_size

        # self.latent_size = latent_size
        self.Inverter_learning_rate = Inverter_learning_rate
        self.fc1 = nn.Linear(31, 128)
        self.fc2 = nn.Linear(128, 256)
        # self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(256, 256)
        self.fc5 = nn.Linear(256, 128)
        self.fc6= nn.Linear(128,56)
        # self.fc4= nn.Linear(64,9)
        # self.fc3 = nn.Linear(128, 32)
        # self.fc4 = nn.Linear(32, 4)
        self.drop = nn.Dropout(p=drop_out_rate)

        self.bn1 = nn.BatchNorm1d(num_features=128, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn2 = nn.BatchNorm1d(num_features=256, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn3 = nn.BatchNorm1d(num_features=512, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn4 = nn.BatchNorm1d(num_features=256, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn5 = nn.BatchNorm1d(num_features=128, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )
        self.bn6 = nn.BatchNorm1d(num_features=31, eps=1e-5, momentum=0.1, affine=False, track_running_stats=True )

        self.prelu1 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu2 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu3 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu4 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu5 = nn.PReLU(num_parameters=1, init=0.25)
        self.prelu6 = nn.PReLU(num_parameters=1, init=0.25)


        # self.softmax = nn.Softmax(dim=-1)
        # self.tanh = nn.Tanh()
        # self.softmax = nn.Softmax(dim=1)


    def forward(self,x):
        x= torch.flatten(x, start_dim=1)
        # print (x.shape)
        x = self.fc1(x)
        x = self.prelu1(x)
        x= self.drop(x)
        x= self.bn1(x)

        x = self.fc2(x)
        x = self.prelu2(x)
        x= self.drop(x)
        x= self.bn2(x)

        # x = self.fc3(x)
        # x = self.prelu3(x)
        # x= self.drop(x)
        # x= self.bn3(x)

        x = self.fc4(x)
        x = self.prelu4(x)
        x= self.drop(x)
        x= self.bn4(x)

        x = self.fc5(x)
        x = self.prelu5(x)
        x= self.drop(x)
        x= self.bn5(x)

        x = self.fc6(x)
        x = self.prelu6(x)
        # x= self.drop(x)
        # x= self.bn6(x)


        # x = self.fc4(x)
        # x = self.softmax(x)
        # print(x.shape)

        # x = self.tanh(x)

        return (x)


In [ ]:
m = Mapper(Mapper_input_size,Mapper_output_size,Mapper_learning_rate)
i = Inverter(Inverter_input_size,Inverter_output_size,Inverter_learning_rate)

In [ ]:
class RegenrativeModel(pl.LightningModule):

    def __init__(
        self,
        lr_m: float = lr_m,
        b1_m: float = b1_m,
        b2_m: float = b2_m,
        lr_i: float = lr_i,
        b1_i: float = b1_i,
        b2_i: float = b2_i,

        batch_size: int = batch_size,
        num_epochs: int = num_epochs,
        **kwargs,):
        super().__init__()
        self.save_hyperparameters()
        self.automatic_optimization = False


        self.mapper = Mapper(Mapper_input_size, Mapper_output_size,  Mapper_learning_rate)
        self.inverter = Inverter(Inverter_input_size, Inverter_output_size,  Inverter_learning_rate)

    def forward(self, data):
      return self.mapper(data)

    def mapper_loss(self,hat,true):
        norm = torch.sqrt(torch.sum(hat ** 2, -1, keepdim=True))
        normalized_emb = hat / norm
        similarity = torch.matmul(normalized_emb, normalized_emb.transpose(1, 0))
        batch_size = hat.size(0)
        pt = (torch.sum(similarity) - batch_size) / (batch_size * (batch_size - 1)) # cos similarity
        hce_loss = havrda_charvat_entropy(hat, true, parameter_a=1.3)
        l1_loss = F.l1_loss(hat, true)
        l2_loss = F.mse_loss(hat, true)
        tji = 1- k_twin(hat.detach().cpu().numpy(), true.detach().cpu().numpy(), 5, "L2")
        loss = l1_lamb * l1_loss + l2_lamb * l2_loss + pt_lamb *pt + hce_lamb * hce_loss + tji_lamb * tji
        self.log("train/mapping_loss", loss, on_step=False, on_epoch=True)
        self.log("train/mapping_l1_loss", l1_loss, on_step=False, on_epoch=True)
        self.log("train/mapping_l2_loss", l2_loss, on_step=False, on_epoch=True)
        self.log("train/mapping_pt_loss", pt, on_step=False, on_epoch=True)
        self.log("train/mapping_hce_loss", hce_loss, on_step=False, on_epoch=True)
        self.log("train/mapping_tji_loss", tji, on_step=False, on_epoch=True)
        return loss

    def inverter_loss(self,hat,true):
        norm = torch.sqrt(torch.sum(hat ** 2, -1, keepdim=True))
        normalized_emb = hat / norm
        similarity = torch.matmul(normalized_emb, normalized_emb.transpose(1, 0))
        batch_size = hat.size(0)
        pt = (torch.sum(similarity) - batch_size) / (batch_size * (batch_size - 1)) # cos similarity
        hce_loss = havrda_charvat_entropy(hat, true, parameter_a=1.3)
        l1_loss = F.l1_loss(hat, true)
        l2_loss = F.mse_loss(hat, true)
        tji = 1 - k_twin(hat.detach().cpu().numpy(), true.detach().cpu().numpy(), 5, "L2")
        loss = l1_lamb * l1_loss + l2_lamb * l2_loss + pt_lamb *pt + hce_lamb * hce_loss + tji_lamb * tji
        self.log("train/inverting_loss", loss, on_step=False, on_epoch=True)
        self.log("train/inverting_l1_loss", l1_loss, on_step=False, on_epoch=True)
        self.log("train/inverting_l2_loss", l2_loss, on_step=False, on_epoch=True)
        self.log("train/inverting_pt_loss", pt, on_step=False, on_epoch=True)
        self.log("train/inverting_hce_loss", hce_loss, on_step=False, on_epoch=True)
        self.log("train/inverting_tji_loss", tji, on_step=False, on_epoch=True)
        return loss


    def training_step(self, batch, batch_idx):
        optimizer_m, optimizer_i = self.optimizers()

        for i in range (0,2):
            #mapper
            self.toggle_optimizer(optimizer_m)
            data, label = batch
            # y_hat = self.mapper(data)
            # y_hat_hat = self.inverter(y_hat)
            # m_loss = self.mapper_loss(y_hat_hat, data)
            x_hat = self.inverter(label)
            x_hat_hat = self.mapper(x_hat)
            i_loss = self.inverter_loss(x_hat_hat, label)
            self.manual_backward(i_loss)
            optimizer_m.step()
            optimizer_m.zero_grad()
            self.untoggle_optimizer(optimizer_m)
        for i in range (0,1):
            #inverter
            self.toggle_optimizer(optimizer_i)
            data, label = batch
            # x_hat = self.inverter(label)
            # x_hat_hat = self.mapper(x_hat)
            # i_loss = self.inverter_loss(x_hat_hat, label)
            y_hat = self.mapper(data)
            y_hat_hat = self.inverter(y_hat)
            m_loss = self.mapper_loss(y_hat_hat, data)
            self.manual_backward(m_loss)
            optimizer_i.step()
            optimizer_i.zero_grad()
            self.untoggle_optimizer(optimizer_i)

        # self.log("train/m_loss", m_loss, on_step=False, on_epoch=True)
        # self.log("train/i_loss", i_loss, on_step=False, on_epoch=True)

    def validation_step(self, batch, batch_idx):
        data, label = batch
        y_hat = self.mapper(data)
        y_hat_hat = self.inverter(y_hat)
        # m_loss = self.mapper_loss(y_hat_hat, data) #dont demethylate the loss definations are defined for training, they log train loss
        m_l1_loss = F.l1_loss(y_hat_hat, data)
        m_l2_loss = F.mse_loss(y_hat_hat, data)

        x_hat = self.inverter(label)
        x_hat_hat = self.mapper(x_hat)
        # i_loss = self.inverter_loss(x_hat_hat, label)
        i_l1_loss = F.l1_loss(x_hat_hat, label)
        i_l2_loss = F.mse_loss(x_hat_hat, label)

        self.log("val/mapping_l1_loss", m_l1_loss, on_step=False, on_epoch=True)
        self.log("val/mapping_l2_loss", m_l2_loss, on_step=False, on_epoch=True)
        # self.log("val/mapping_loss", m_loss, on_step=False, on_epoch=True)

        self.log("val/inverting_l1_loss", i_l1_loss, on_step=False, on_epoch=True)
        self.log("val/inverting_l2_loss", i_l2_loss, on_step=False, on_epoch=True)
        # self.log("val/inverting_loss", i_loss, on_step=False, on_epoch=True)

        # pca = PCA(n_components=2)
        # pca_mox = pca.fit_transform(y_hat.cpu().numpy())
        # pca_iomox = pca.fit_transform(y_hat_hat.cpu().numpy())
        # pca_ioy = pca.fit_transform(x_hat.cpu().numpy())
        # pca_moxioy = pca.fit_transform(x_hat_hat.cpu().numpy())

        # plt.close()

        # fig, axs = plt.subplots(2, 2, figsize=(10, 8))
        # axs[0, 0].scatter(pca_mox[:, 0], pca_mox[:, 1],s=1)
        # axs[0, 0].set_title('m(X)')
        # axs[0, 1].scatter(pca_iomox[:, 0], pca_mox[:, 1],s=1)
        # axs[0, 1].set_title('i(m(X))')
        # axs[1, 0].scatter(pca_mox[:, 0], pca_mox[:, 1],s=1)
        # axs[1, 0].set_title('i(Y)')
        # axs[1, 1].scatter(pca_mox[:, 0], pca_mox[:, 1],s=1)
        # axs[1, 1].set_title('m(i(Y))')


    def configure_optimizers(self):
        lr_m = self.hparams.lr_m
        b1_m = self.hparams.b1_m
        b2_m = self.hparams.b2_m
        lr_i = self.hparams.lr_i
        b1_i = self.hparams.b1_i
        b2_i = self.hparams.b2_i

        opt_m = torch.optim.Adam(self.mapper.parameters(), lr=lr_m, betas=(b1_m, b2_m))
        opt_i = torch.optim.Adam(self.inverter.parameters(), lr=lr_i, betas=(b1_i, b2_i))
        return [opt_m, opt_i], []

    # def on_validation_epoch_end(self):
    #     # data, label = batch
    #     # y_hat = self.mapper(data)
    #     # y_hat_hat = self.inverter(y_hat)
    #     # m_loss = self.mapper_loss(y_hat_hat, data)
    #     # x_hat = self.inverter(label)
    #     # x_hat_hat = self.mapper(x_hat)

    #     # pca = PCA(n_components=2)
    #     # pca_mox = pca.fit_transform(y_hat.cpu().numpy())
    #     # pca_iomox = pca.fit_transform(y_hat_hat.cpu().numpy())
    #     # pca_ioy = pca.fit_transform(x_hat.cpu().numpy())
    #     # pca_moxioy = pca.fit_transform(x_hat_hat.cpu().numpy())

    #     # fig, axs = plt.subplots(2, 2, figsize=(10, 8))
    #     # axs[0, 0].scatter(pca_mox[:, 0], pca_mox[:, 1])
    #     # axs[0, 0].set_title('m(X)')
    #     # axs[0, 0].scatter(pca_iomox[:, 0], pca_mox[:, 1])
    #     # axs[0, 0].set_title('i(m(X))')
    #     # axs[0, 0].scatter(pca_mox[:, 0], pca_mox[:, 1])
    #     # axs[0, 0].set_title('i(Y)')
    #     # axs[0, 0].scatter(pca_mox[:, 0], pca_mox[:, 1])
    #     # axs[0, 0].set_title('m(i(Y))')
    #     print("...")

    def on_save_checkpoint(self, checkpoint):
        checkpoint["hyperparameters"] = ( num_epochs, batch_size, Mapper_input_size, Mapper_output_size,  Mapper_learning_rate, Inverter_input_size, Inverter_output_size,  Inverter_learning_rate)

    def on_load_checkpoint(self, checkpoint):
        num_epochs, batch_size, Mapper_input_size, Mapper_output_size,  Mapper_learning_rate, Inverter_input_size, Inverter_output_size,  Inverter_learning_rate = checkpoint["hyperparameters"]

# Traning

In [ ]:
# ! mkdir /content/drive/MyDrive/KTI
# ! mkdir /content/drive/MyDrive/KTI/Model
! mkdir /content/drive/MyDrive/KTI/Model/ktir6_1
! mkdir /content/drive/MyDrive/KTI/Model/ktir6_1/m0t0

In [ ]:
model = RegenrativeModel()

wandb_logger = WandbLogger(name='ktir6_1-m0t0',project='KTI')

wandb_logger.watch(model, log="all", log_graph=True) # log frequency != num epochs to print loos rather num steps

checkpoint_callback = ModelCheckpoint(dirpath='/content/drive/MyDrive/KTI/Model/ktir6_1/m0t0',
                                      filename='{epoch}-{training_loss:.2f}-{val_loss:.2f}',
                                      # monitor="valid/val_loss", mode = "min",
                                    #   every_n_train_steps=every_n_train_steps,
                                    #   every_n_epochs= 25
                                      verbose=True,
                                      # save_on_train_epoch_end=True,
                                      save_last = True,
                                      every_n_epochs= 100,
                                      save_top_k=-1
                                      )

trainer = pl.Trainer(max_epochs=num_epochs,
                     callbacks=[ checkpoint_callback],
                     gradient_clip_val=grad_clip,
                     accelerator="gpu", #amp_backend="native")
                     val_check_interval=2,
                     logger=wandb_logger)

In [ ]:
trainer.fit(model, train_dataloaders=training_loader, val_dataloaders= validation_loader)

In [ ]:
# if torch.isinf(x_train_tt).any():
#     print("Data contains Infs")

In [ ]:
# if torch.isnan(x_train_tt).any():
#     print("Data contains NaNs ")

# Test

## ktri1-0/m0t0/epoch=599

### Chose model

In [ ]:
checkpoint_path= "/content/drive/MyDrive/KTI/Model/ktir6_0/m0t0/epoch=1199-training_loss=0.00-val_loss=0.00.ckpt"

### Loading

In [ ]:
# Mapper Loading

ckpt_kwargs = {
    'Mapper_input_size': Mapper_input_size,
    'Mapper_output_size': Mapper_output_size,
    'Mapper_learning_rate': Mapper_learning_rate,
    # 'map_location': torch.device('cpu')  # Add map_location here if needed
}
# Mapper = Mapper.load_from_checkpoint(checkpoint, **ckpt_kwargs)
# Load the checkpoint state_dict
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
state_dict = checkpoint["state_dict"]

# Create a new state_dict with modified keys
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith("mapper."):
        new_state_dict[k[len("mapper."):]] = v  # Remove the "mapper." prefix
    else:
        # Handle other keys if needed (e.g., optimizer state)
        # or raise an error if they are unexpected
        pass

# Load the modified state_dict into your model
m = Mapper(**ckpt_kwargs)  # Assuming Mapper is your model class
m.load_state_dict(new_state_dict, strict=False) # strict=False handles missing or extra keys


In [ ]:
# Inverter Loading

ckpt_kwargs = {
    'Inverter_input_size': Inverter_input_size,
    'Inverter_output_size': Inverter_output_size,
    'Inverter_learning_rate': Inverter_learning_rate,
    # 'map_location': torch.device('cpu')  # Add map_location here if needed
}
# Inverter = Inverter.load_from_checkpoint(checkpoint, **ckpt_kwargs)
# Load the checkpoint state_dict
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
state_dict = checkpoint["state_dict"]

# Create a new state_dict with modified keys
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith("inverter."):
        new_state_dict[k[len("inverter."):]] = v  # Remove the "mapper." prefix
    else:
        # Handle other keys if needed (e.g., optimizer state)
        # or raise an error if they are unexpected
        pass

# Load the modified state_dict into your model
i = Inverter(**ckpt_kwargs)  # Assuming Inverter is your model class
i.load_state_dict(new_state_dict, strict=False) # strict=False handles missing or extra keys


### Train

In [ ]:
# (np.sum(np.abs
#  (  x_train_tt.detach().numpy() - i(m(x_train_tt)).detach().numpy()   )
#  ))/ (x_train.shape[0]*x_train.shape[1])

In [ ]:
# (np.sum(np.abs
#  (  x_test_tt.detach().numpy() - i(m(x_test_tt)).detach().numpy()   )
#  ))/ (x_test.shape[0]*x_test.shape[1])

In [ ]:
 y_hat_hat_train = i(m(x_train_tt)).detach().cpu().numpy()

In [ ]:
n_components = 3
pca = PCA(n_components=n_components)
components = pca.fit_transform((x_train))

fig = px.scatter_matrix(
    components,
    # color=score,
    # labels=anno_mouse_2.Sample_ID,
    # symbol=anno_mouse_2.Material,
    dimensions=range(n_components),
    # title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()
fig1 = go.Figure()
fig1.add_trace(go.Heatmap(z=pd.DataFrame(x_train).corr(), colorscale='Puor', showscale=True, zmin=-1, zmax=1,))
fig1.update_layout(height=500, width=500)
fig1.show()

In [ ]:
fig = go.Figure(data=go.Scatter(
    x=components[:,0],
    y=components[:,1],
    mode='markers',
    marker=dict(
        size=5,  # Big marker size
        color='blue'
    )
))

# Customize the layout
fig.update_layout(
    title='Pred-SHARE-oid by ktr0.2-m2t0e1600',
    xaxis_title='PC 1',
    yaxis_title='PC 2',

    height = 600,
    width = 750,
    plot_bgcolor='white',  # Background color white

    xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
    yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

)

# Display the scatter plot
fig.show()

In [ ]:
print("norm_L1_error:", (np.sum(np.abs(x_train - y_hat_hat_train)))/(y_hat_hat_train.shape[0]*y_hat_hat_train.shape[1]) )
print("norm_L2_error:", ((np.sum((x_train - y_hat_hat_train)**2))**0.5)/(y_hat_hat_train.shape[0]*y_hat_hat_train.shape[1]) )


n_components = 3
pca = PCA(n_components=n_components)
components = pca.fit_transform((y_hat_hat_train))

fig = px.scatter_matrix(
    components,
    # color=score,
    # labels=anno_mouse_2.Sample_ID,
    # symbol=anno_mouse_2.Material,
    dimensions=range(n_components),
    # title=f'Total Explained Variance: {total_var:.2f}%',
)
fig.update_traces(diagonal_visible=False)
fig.show()
fig1 = go.Figure()
fig1.add_trace(go.Heatmap(z=pd.DataFrame(y_hat_hat_train).corr(), colorscale='Puor', showscale=True, zmin=-1, zmax=1,))
fig1.update_layout(height=500, width=500)
fig1.show()

In [ ]:
fig = go.Figure(data=go.Scatter(
    x=components[:,0],
    y=components[:,1],
    mode='markers',
    marker=dict(
        size=5,  # Big marker size
        color='blue'
    )
))

# Customize the layout
fig.update_layout(
    title='Pred-SHARE-oid by ktr0.2-m2t0e1600',
    xaxis_title='PC 1',
    yaxis_title='PC 2',

    height = 600,
    width = 750,
    plot_bgcolor='white',  # Background color white

    xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
    yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

)

# Display the scatter plot
fig.show()

### Test

In [ ]:
# (np.sum(np.abs
#  (  x_train_tt.detach().numpy() - i(m(x_train_tt)).detach().numpy()   )
#  ))/ (x_train.shape[0]*x_train.shape[1])

In [ ]:
# (np.sum(np.abs
#  (  x_test_tt.detach().numpy() - i(m(x_test_tt)).detach().numpy()   )
#  ))/ (x_test.shape[0]*x_test.shape[1])

### mapper

In [ ]:
 y_hat_hat_test = i(m(x_test_tt)).detach().cpu().numpy()

In [ ]:
# n_components = 3
# pca = PCA(n_components=n_components)
# components = pca.fit_transform((x_test))

# fig = px.scatter_matrix(
#     components,
#     # color=score,
#     # labels=anno_mouse_2.Sample_ID,
#     # symbol=anno_mouse_2.Material,
#     dimensions=range(n_components),
#     # title=f'Total Explained Variance: {total_var:.2f}%',
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()
# fig1 = go.Figure()
# fig1.add_trace(go.Heatmap(z=pd.DataFrame(x_test).corr(), colorscale='Puor', showscale=True, zmin=-1, zmax=1,))
# fig1.update_layout(height=500, width=500)
# fig1.show()

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=components[:,0],
#     y=components[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='Pred-SHARE-oid by ktr0.2-m2t0e1600',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# print("norm_L1_error:", (np.sum(np.abs(x_test - y_hat_hat_test)))/(y_hat_hat_test.shape[0]*y_hat_hat_test.shape[1]) )
# print("norm_L2_error:", ((np.sum((x_test - y_hat_hat_test)**2))**0.5)/(y_hat_hat_test.shape[0]*y_hat_hat_test.shape[1]) )


# n_components = 3
# pca = PCA(n_components=n_components)
# components = pca.fit_transform((y_hat_hat_test))

# fig = px.scatter_matrix(
#     components,
#     # color=score,
#     # labels=anno_mouse_2.Sample_ID,
#     # symbol=anno_mouse_2.Material,
#     dimensions=range(n_components),
#     # title=f'Total Explained Variance: {total_var:.2f}%',
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()
# fig1 = go.Figure()
# fig1.add_trace(go.Heatmap(z=pd.DataFrame(y_hat_hat_test).corr(), colorscale='Puor', showscale=True, zmin=-1, zmax=1,))
# fig1.update_layout(height=500, width=500)
# fig1.show()

In [ ]:
# y_hat_hat_rand_test = np.random.rand(x_test.shape[0], x_test.shape[1])
# y_hat_hat_test_norm = (y_hat_hat_rand_test-y_hat_hat_rand_test.min())/(y_hat_hat_rand_test.max()-y_hat_hat_rand_test.min())
# print("random_norm_L1_error:", (np.sum(np.abs(x_test - y_hat_hat_rand_test)))/(y_hat_hat_rand_test.shape[0]*y_hat_hat_rand_test.shape[1]) )
# print("random_norm_L2_error:", ((np.sum((x_test - y_hat_hat_rand_test)**2))**0.5)/(y_hat_hat_rand_test.shape[0]*y_hat_hat_rand_test.shape[1]) )

In [ ]:
target = x_test
predicted = y_hat_hat_test

# Choose distance metric ('L1' or 'L2')
distance_metric = 'L1'

# Function to calculate distance
def calculate_distance(a, b, metric):
    if metric == 'L1':
        return (np.sum(np.abs(a - b)))/(a.shape[0]*a.shape[1])
    # elif metric == 'L2':
    #     return euclidean(a, b)
    else:
        raise ValueError("Invalid distance metric. Choose 'L1' or 'L2'.")

# Calculate distance for the original predicted
original_distance = calculate_distance(target, predicted, distance_metric)

# Calculate distances for 1000 random shuffles of predicted
random_distances = []
for _ in range(1000):
    rand_prediction = np.random.rand(predicted.shape[0], predicted.shape[1])
    rand_prediction_norm = (rand_prediction-rand_prediction.min())/(rand_prediction.max()-rand_prediction.min())

    distance = calculate_distance(target, rand_prediction_norm, distance_metric)
    random_distances.append(distance)

# Calculate p-value
p_value = np.sum(np.array(random_distances) <= original_distance) / len(random_distances)

print(f"Original {distance_metric} distance: {original_distance:.2f}")
print(f"Mean random {distance_metric} distance: {np.mean(random_distances):.2f}")
print(f"P-value: {p_value:.7f}")

In [ ]:
target = x_test
predicted = y_hat_hat_test

# Choose distance metric ('L1' or 'L2')
distance_metric = 'L1'

# Function to calculate distance
def calculate_distance(a, b, metric):
    if metric == 'L1':
        return (np.sum(np.abs(a - b)))/(a.shape[0]*a.shape[1])
    # elif metric == 'L2':
    #     return euclidean(a, b)
    else:
        raise ValueError("Invalid distance metric. Choose 'L1' or 'L2'.")

# Calculate distance for the original predicted
original_distance = calculate_distance(target, predicted, distance_metric)

# Calculate distances for 1000 random shuffles of predicted
random_distances = []

shuffled_pred = predicted.copy()
for _ in range(1000):
    # rand_prediction = np.random.rand(predicted.shape[0], predicted.shape[1])
    # rand_prediction_norm = (rand_prediction-rand_prediction.min())/(rand_prediction.max()-rand_prediction.min())
    np.random.shuffle(shuffled_pred)

    distance = calculate_distance(target, shuffled_pred, distance_metric)
    random_distances.append(distance)

# Calculate p-value
p_value = np.sum(np.array(random_distances) <= original_distance) / len(random_distances)

print(f"Original {distance_metric} distance: {original_distance:.2f}")
print(f"Mean random {distance_metric} distance: {np.mean(random_distances):.2f}")
print(f"P-value: {p_value:.7f}")

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=components[:,0],
#     y=components[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='i(m(A_test))',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
# umap_x = umap_model.fit_transform(x_test)
# umap_y_hat_hat = umap_model.fit_transform(y_hat_hat_test)

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=umap_x[:,0],
#     y=umap_x[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='A_test',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=umap_y_hat_hat[:,0],
#     y=umap_y_hat_hat[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='i(m(A_test))',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
per = 10
k = (x_test.shape[0]*per)//100
k

In [ ]:
components_a = x_test
components_b = y_hat_hat_test

x = (1, 5, 10, 15, 20, 30, 31, 40, 50, 55)
y = np.zeros_like(x)
for i,x_i in enumerate(x):
    # y[i]= k_twin(components_a,components_b, x_i, "L1")
    print ("for ", x_i, ":", k_twin(components_a,components_b, x_i, "L1"))

# print("-----")

# rnd = np.zeros_like(x)
# np.random.shuffle(components_b)
# for i,x_i in enumerate(x):
#     # rnd[i]= k_twin(components_a,components_b, x_i, "L2")
#     print (k_twin(components_a,components_b, x_i, "L2"))

In [ ]:
# components_a = x_test
# components_b = y_hat_hat_test

# x = np.arange (1,20, 10)
# y = np.zeros_like(x)
# for i,x_i in enumerate(x):
#     # y[i]= k_twin(components_a,components_b, x_i, "L1")
#     print (k_twin(components_a,components_b, x_i, "L1"))

# print("-----")

# rnd = np.zeros_like(x)
# np.random.shuffle(components_b)
# for i,x_i in enumerate(x):
#     # rnd[i]= k_twin(components_a,components_b, x_i, "L2")
#     print (k_twin(components_a,components_b, x_i, "L2"))

In [ ]:
# components_a = umap_x
# components_b = umap_y_hat_hat

# x = np.arange (1,20, 10)
# y = np.zeros_like(x)
# for i,x_i in enumerate(x):
#     # y[i]= k_twin(components_a,components_b, x_i, "L1")
#     print (k_twin(components_a,components_b, x_i, "L1"))

# print("-----")

# rnd = np.zeros_like(x)
# np.random.shuffle(components_b)
# for i,x_i in enumerate(x):
#     # rnd[i]= k_twin(components_a,components_b, x_i, "L2")
#     print (k_twin(components_a,components_b, x_i, "L2"))

In [ ]:
# rips = Rips(maxdim=2, thresh= 0.9)
# data = x_test
# diagrams_b = rips.fit_transform(data)
# rips.plot(diagrams_b)

In [ ]:
# rips = Rips(maxdim=2, thresh= 0.9)
# data = y_hat_hat_test
# diagrams_b = rips.fit_transform(data)
# rips.plot(diagrams_b)

### inverter

In [ ]:
x_hat_hat_test = m(i(y_test_tt)).detach().cpu().numpy()

In [ ]:
# n_components = 3
# pca = PCA(n_components=n_components)
# components = pca.fit_transform((y_test))

# fig = px.scatter_matrix(
#     components,
#     # color=score,
#     # labels=anno_mouse_2.Sample_ID,
#     # symbol=anno_mouse_2.Material,
#     dimensions=range(n_components),
#     # title=f'Total Explained Variance: {total_var:.2f}%',
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()
# fig1 = go.Figure()
# fig1.add_trace(go.Heatmap(z=pd.DataFrame(x_test).corr(), colorscale='Puor', showscale=True, zmin=-1, zmax=1,))
# fig1.update_layout(height=500, width=500)
# fig1.show()

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=components[:,0],
#     y=components[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='Pred-SHARE-oid by ktr0.2-m2t0e1600',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# print("norm_L1_error:", (np.sum(np.abs(y_test - x_hat_hat_test)))/(x_hat_hat_test.shape[0]*x_hat_hat_test.shape[1]) )
# print("norm_L2_error:", ((np.sum((y_test - x_hat_hat_test)**2))**0.5)/(x_hat_hat_test.shape[0]*x_hat_hat_test.shape[1]) )


# n_components = 3
# pca = PCA(n_components=n_components)
# components = pca.fit_transform((x_hat_hat_test))

# fig = px.scatter_matrix(
#     components,
#     # color=score,
#     # labels=anno_mouse_2.Sample_ID,
#     # symbol=anno_mouse_2.Material,
#     dimensions=range(n_components),
#     # title=f'Total Explained Variance: {total_var:.2f}%',
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()
# fig1 = go.Figure()
# fig1.add_trace(go.Heatmap(z=pd.DataFrame(x_hat_hat_test).corr(), colorscale='Puor', showscale=True, zmin=-1, zmax=1,))
# fig1.update_layout(height=500, width=500)
# fig1.show()

In [ ]:
# x_hat_hat_rand_test = np.random.rand(y_test.shape[0], y_test.shape[1])
# x_hat_hat_test_norm = (x_hat_hat_rand_test-x_hat_hat_rand_test.min())/(x_hat_hat_rand_test.max()-x_hat_hat_rand_test.min())
# print("random_norm_L1_error:", (np.sum(np.abs(y_test - x_hat_hat_rand_test)))/(x_hat_hat_rand_test.shape[0]*x_hat_hat_rand_test.shape[1]) )
# print("random_norm_L2_error:", ((np.sum((y_test - x_hat_hat_rand_test)**2))**0.5)/(x_hat_hat_rand_test.shape[0]*x_hat_hat_rand_test.shape[1]) )

In [ ]:
target = y_test
predicted = x_hat_hat_test

# Choose distance metric ('L1' or 'L2')
distance_metric = 'L1'

# Function to calculate distance
def calculate_distance(a, b, metric):
    if metric == 'L1':
        return (np.sum(np.abs(a - b)))/(a.shape[0]*a.shape[1])
    # elif metric == 'L2':
    #     return euclidean(a, b)
    else:
        raise ValueError("Invalid distance metric. Choose 'L1' or 'L2'.")

# Calculate distance for the original predicted
original_distance = calculate_distance(target, predicted, distance_metric)

# Calculate distances for 1000 random shuffles of predicted
random_distances = []
for _ in range(1000):
    rand_prediction = np.random.rand(predicted.shape[0], predicted.shape[1])
    rand_prediction_norm = (rand_prediction-rand_prediction.min())/(rand_prediction.max()-rand_prediction.min())

    distance = calculate_distance(target, rand_prediction_norm, distance_metric)
    random_distances.append(distance)

# Calculate p-value
p_value = np.sum(np.array(random_distances) <= original_distance) / len(random_distances)

print(f"Original {distance_metric} distance: {original_distance:.2f}")
print(f"Mean random {distance_metric} distance: {np.mean(random_distances):.2f}")
print(f"P-value: {p_value:.7f}")

In [ ]:
target = y_test
predicted = x_hat_hat_test

# Choose distance metric ('L1' or 'L2')
distance_metric = 'L1'

# Function to calculate distance
def calculate_distance(a, b, metric):
    if metric == 'L1':
        return (np.sum(np.abs(a - b)))/(a.shape[0]*a.shape[1])
    # elif metric == 'L2':
    #     return euclidean(a, b)
    else:
        raise ValueError("Invalid distance metric. Choose 'L1' or 'L2'.")

# Calculate distance for the original predicted
original_distance = calculate_distance(target, predicted, distance_metric)

# Calculate distances for 1000 random shuffles of predicted
random_distances = []

shuffled_pred = predicted.copy()
for _ in range(1000):
    # rand_prediction = np.random.rand(predicted.shape[0], predicted.shape[1])
    # rand_prediction_norm = (rand_prediction-rand_prediction.min())/(rand_prediction.max()-rand_prediction.min())
    np.random.shuffle(shuffled_pred)

    distance = calculate_distance(target, shuffled_pred, distance_metric)
    random_distances.append(distance)

# Calculate p-value
p_value = np.sum(np.array(random_distances) <= original_distance) / len(random_distances)

print(f"Original {distance_metric} distance: {original_distance:.2f}")
print(f"Mean random {distance_metric} distance: {np.mean(random_distances):.2f}")
print(f"P-value: {p_value:.7f}")

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=components[:,0],
#     y=components[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='m(i(B_test))',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# umap_model = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2)
# umap_x = umap_model.fit_transform(y_test)
# umap_y_hat_hat = umap_model.fit_transform(x_hat_hat_test)

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=umap_x[:,0],
#     y=umap_x[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='B_test',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# fig = go.Figure(data=go.Scatter(
#     x=umap_y_hat_hat[:,0],
#     y=umap_y_hat_hat[:,1],
#     mode='markers',
#     marker=dict(
#         size=5,  # Big marker size
#         color='blue'
#     )
# ))

# # Customize the layout
# fig.update_layout(
#     title='m(i(B_test))',
#     xaxis_title='PC 1',
#     yaxis_title='PC 2',

#     height = 600,
#     width = 750,
#     plot_bgcolor='white',  # Background color white

#     xaxis=dict(showgrid=True, gridcolor='black',zeroline = True, zerolinecolor="black", showline= True, linecolor="black"),
#     yaxis=dict(showgrid=True,  gridcolor='black',zeroline = True, zerolinecolor="black",showline= True, linecolor="black"),

# )

# # Display the scatter plot
# fig.show()

In [ ]:
# components_a = y_test
# components_b = x_hat_hat_test

# x = np.arange (1,20, 10)
# y = np.zeros_like(x)
# for i,x_i in enumerate(x):
#     # y[i]= k_twin(components_a,components_b, x_i, "L1")
#     print (k_twin(components_a,components_b, x_i, "L1"))

# print("-----")

# rnd = np.zeros_like(x)
# np.random.shuffle(components_b)
# for i,x_i in enumerate(x):
#     # rnd[i]= k_twin(components_a,components_b, x_i, "L2")
#     print (k_twin(components_a,components_b, x_i, "L2"))

In [ ]:
# components_a = umap_x
# components_b = umap_y_hat_hat

# x = np.arange (1,20, 10)
# y = np.zeros_like(x)
# for i,x_i in enumerate(x):
#     # y[i]= k_twin(components_a,components_b, x_i, "L1")
#     print (k_twin(components_a,components_b, x_i, "L1"))

# print("-----")

# rnd = np.zeros_like(x)
# np.random.shuffle(components_b)
# for i,x_i in enumerate(x):
#     # rnd[i]= k_twin(components_a,components_b, x_i, "L2")
#     print (k_twin(components_a,components_b, x_i, "L2"))

In [ ]:
components_a = y_test
components_b = x_hat_hat_test

x = (1, 5, 10, 15, 20, 30, 31, 40, 50, 55)
y = np.zeros_like(x)
for i,x_i in enumerate(x):
    # y[i]= k_twin(components_a,components_b, x_i, "L1")
    print ("for ", x_i, ":", k_twin(components_a,components_b, x_i, "L1"))

# print("-----")

# rnd = np.zeros_like(x)
# np.random.shuffle(components_b)
# for i,x_i in enumerate(x):
#     # rnd[i]= k_twin(components_a,components_b, x_i, "L2")
#     print (k_twin(components_a,components_b, x_i, "L2"))

In [ ]:
rips = Rips(maxdim=2, thresh= 0.9)
data = x_test
diagrams_b = rips.fit_transform(data)
rips.plot(diagrams_b)

In [ ]:
rips = Rips(maxdim=2, thresh= 0.9)
data = y_hat_hat_test
diagrams_b = rips.fit_transform(data)
rips.plot(diagrams_b)